In [10]:
# *********************
# *** Setup Section ***
# *********************

import pandas as pd

# Emtion DataFram
# This was predtermined by review of the emotion from English to German
emotion_key = {
    "boredom": "---",
    "love": "Vertrauen",
    "relief": "---",
    "fun": "---",
    "hate": "Ekel",
    "neutral": "Unklar",
    "anger": "Ärger",
    "happiness": "Freude",
    "surprise": "Überraschung",
    "sadness": "Traurigkeit",
    "worry": "Angst",
    "enthusiasm": "Antizipation",
    "empty ": "---",
    "---": "Keine"
}

# Create a DataFrame from the emotion_key dictionary
df_emotions = pd.DataFrame(emotion_key.items(), columns=['emotion_en', 'emotion_de'])


# *******************
# *** Translation ***
# *******************
from googletrans import Translator
import time

# This google API take a Sentence and converts to German
def translate(sentence, dest_lang):
    try:
        translator = Translator()
        translator.raise_Exception = True
        translation = translator.translate(sentence, dest=dest_lang)
        time.sleep(0.5)  # Add a 1-second delay
        return translation.text
    except Exception as e:
        print(f"Translation Error: {e}")
        return None


In [7]:
# *********************************
# *** English Data Preparations ***
# *********************************

import pandas as pd
from tqdm import tqdm


# Load English CSV File into a Dataframe
df_en = pd.read_csv("data/english.csv")
print(f'Count (raw): {len(df_en)}')

# Print Columns
print(f'English Source Columns: {df_en.columns}\n')

# Filter only labels that we matched with the German counter part JSON File
filter_values = ["love", "hate", "neutral", "anger", "happiness", "surprise", "sadness", "worry", "enthusiasm"]
df_en = df_en[df_en["sentiment"].isin(filter_values)]

# Find Empty Sentence Values
df_en = df_en[df_en["content"].notnull() & (df_en["content"].str.strip() != '')]

# Remove unnecessary column
df_en = df_en.drop("tweet_id", axis=1)

# Rename Columns
df_en = df_en.rename(columns = {'sentiment': 'emotion_en', 'content': 'sentence_en'})

# Merge German Emotions onto English
df_en = pd.merge(df_en, df_emotions, on='emotion_en', how='left')

# Add German Sentence Column
df_en["sentence_de"] = ""

# Randomly select 1500 rows
df_en = df_en.sample(n=1500, random_state=2023)

df_en.to_csv('./data/pd_en.csv', index=False)


# Iterate over the rows with tqdm to show the progress
for index, row in tqdm(df_en.iterrows(), total=df_en.shape[0]):
    # Call Translation
    sentence = translate(row["sentence_en"], 'de')
    # Save Sentence on Column
    df_en.at[index, 'sentence_de'] = sentence


df_en.to_csv('./data/pd_en_translated.csv', index=False)

Count (raw): 40000
English Source Columns: Index(['tweet_id', 'sentiment', 'content'], dtype='object')



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [1:07:57<00:00,  2.72s/it]


In [11]:
# *********************************
# *** German Data Preparations ***
# *********************************

import pandas as pd
from tqdm import tqdm


# Load German JSON File into a DataFrame
df_de = pd.read_json("data/german.json", lines=True)
print(f'Count (raw): {len(df_de)}')

# Print Columns
print(f'German Source Columns: {df_de.columns}\n')

# Explode the "article_emotion" column to distinct rows
df_de = df_de.explode('article_emotion')

# Filter only labels that we matched with the German counter part JSON File
filter_values = ["Vertrauen","Ekel", "Unklar", "Ärger", "Freude", "Überraschung", "Traurigkeit", "Angst", "Antizipation"]
df_de = df_de[df_de["article_emotion"].isin(filter_values)]

# Find Empty Sentence Values
df_de = df_de[df_de["title"].notnull() & (df_de["title"].str.strip() != '')]

# Remove unnecessary columns
df_de = df_de.drop("article_id", axis=1)
df_de = df_de.drop("article_stance", axis=1)
df_de = df_de.drop("paragraphs", axis=1)
df_de = df_de.drop("source", axis=1)
df_de = df_de.drop("snippet", axis=1)

# Rename Columns
df_de = df_de.rename(columns={'title': 'sentence_de'})
df_de = df_de.rename(columns={'article_emotion': 'emotion_de'})

# Merge Englsih Emotions onto German
df_de = pd.merge(df_de, df_emotions, on='emotion_de', how='left')

# Add English Sentence Column
df_de["sentence_en"] = ""

# Randomly select 1500 rows
df_de = df_de.sample(n=1500, random_state=2023)

df_de.to_csv('./data/pd_de.csv', index=False)


# Iterate over the rows with tqdm to show the progress
for index, row in tqdm(df_de.iterrows(), total=df_de.shape[0]):
    # Call Translation
    sentence = translate(row["sentence_de"], 'en')
    # Save Sentence on Column
    df_de.at[index, 'sentence_en'] = sentence


df_de.to_csv('./data/pd_de_translated.csv', index=False)

Count (raw): 1970
German Source Columns: Index(['article_id', 'title', 'snippet', 'source', 'article_stance',
       'article_emotion', 'paragraphs'],
      dtype='object')



 39%|██████████████████████████████████████████████████████                                                                                     | 583/1500 [11:24<1:21:22,  5.32s/it]

Translation Error: The read operation timed out


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                       | 1252/1500 [28:47<16:11,  3.92s/it]

Translation Error: _ssl.c:980: The handshake operation timed out


...

 89%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎              | 1342/1500 [33:56<15:44,  5.98s/it]

Translation Error: The read operation timed out


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████              | 1351/1500 [34:30<10:47,  4.34s/it]

Translation Error: _ssl.c:980: The handshake operation timed out


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [39:01<00:00,  1.56s/it]


In [5]:
translate("Hallo", 'en')

'Hallo'